In [ ]:
import sys
sys.path.append('../src')
import os
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from preprocessing \
    import WoowahanProcessor, TossProcessor, MediumProcessor, KakaoProcessor, OliveYoungProcessor
from utils.db_conn import Connection
import warnings
import boto3

os.environ["OPENAI_API_KEY"] = boto3.client("ssm", region_name="ap-northeast-2").\
                                    get_parameter(Name='/llm/apikey', WithDecryption=True)\
                                    ["Parameter"]["Value"]
warnings.filterwarnings("ignore")
conn = Connection()

In [ ]:
query = f"""
SELECT
    a.title,
    a.description, 
    a.keywords,
    a.content,
    a.lang
FROM 
    article AS a
JOIN (
    SELECT article_id FROM article ORDER BY RAND() LIMIT 5
) AS rand_table
ON a.article_id = rand_table.article_id;
"""

sample_text = conn.execute(query)

In [ ]:
sample_text['keywords']

In [ ]:
def postprocess_by_blog_id(text, blog_id):
    processors = {
        1: WoowahanProcessor,
        2: TossProcessor,
        3: MediumProcessor,
        4: KakaoProcessor,
        5: OliveYoungProcessor
    }
    processor_class = processors.get(blog_id)
    if not processor_class:
        raise ValueError(f"Unsupported blog_id: {blog_id}")
    processor = processor_class(text, blog_id)
    return processor.process()

In [ ]:
text = sample_text['content'].loc[0]
blog_id = sample_text['blog_id'].loc[0]
text = postprocess_by_blog_id(text, blog_id)

In [ ]:
from llm_pipeline import LangChainModel
model = LangChainModel()

In [ ]:
text

In [ ]:
model.predict(text)

In [ ]:
query = f"""
desc article;
"""

conn.execute(query)

In [ ]:
conn.close()